### AirBnB Montréal
#### SQL Project

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3

### Goals:

#### 1. To know how much the top Airbnb Earners are making in Montréal.
#### 2. To get a list of potential customers for an Airbnb cleaning business.

### Importing Data

The detailed listing data was collected through [Inside AirBnB](http://insideairbnb.com/get-the-data/) on September 12, 2022.
<br> The original dataset contained 13,621 rows and 75 columns but after cleaning the data, there are 13,599 rows and 67 columns.

In [2]:
df = pd.read_csv('../Airbnb_Montreal/cleaned_airbnb.csv')
conn = sqlite3.connect('df.sqlite')
df.to_sql('df',con=conn,index=False,if_exists='replace')
df.head()

/Users/sarbjitmadra/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,host_id,host_url,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,2843,https://www.airbnb.com/rooms/2843,20220912200208,2022-09-12,city scrape,Fabulous downtown/airport room,Come stay in this trendy area of downtown Mont...,2319,https://www.airbnb.com/users/show/2319,...,4.61,4.73,4.73,4.64,t,2,1,1,0,0.97
1,1,29059,https://www.airbnb.com/rooms/29059,20220912200208,2022-09-13,city scrape,Lovely studio Quartier Latin,CITQ 267153 br Lovely studio with 1 closed r...,125031,https://www.airbnb.com/users/show/125031,...,4.81,4.78,4.81,4.71,f,2,2,0,0,2.58
2,2,298059,https://www.airbnb.com/rooms/298059,20220912200208,2022-09-13,city scrape,Superb cottage Mile-End Plateau !,b The space b br Superb bright cottage de...,1536474,https://www.airbnb.com/users/show/1536474,...,4.96,5.00,4.63,4.84,f,1,1,0,0,0.40
3,3,29061,https://www.airbnb.com/rooms/29061,20220912200208,2022-09-13,city scrape,Maison historique - Quartier Latin,Lovely historic house with plenty of period ch...,125031,https://www.airbnb.com/users/show/125031,...,4.85,4.79,4.88,4.71,f,2,2,0,0,0.84
4,4,309367,https://www.airbnb.com/rooms/309367,20220912200208,2022-09-13,city scrape,Charming and Clean ! 5 min Métro,Feel like home in a dynamic area close to eve...,1562348,https://www.airbnb.com/users/show/1562348,...,4.86,4.91,4.32,4.59,f,3,3,0,0,0.18


### Top 20 Airbnb Earners
The *booked_out_30* column represents the numbers of days the accommodation is already reserved for the month.
<br>The *proj_rev_30* column represents the projected income during the reserved days.

In [3]:
pd.read_sql_query("""SELECT id, listing_url, name, 30- availability_30 AS booked_out_30,
                  price * (30 - availability_30) AS proj_rev_30
                  FROM df
                  ORDER BY proj_rev_30 DESC LIMIT 10""", conn)

,id,listing_url,name,booked_out_30,proj_rev_30
0,53088650,https://www.airbnb.com/rooms/53088650,"room in a shared apartment, the (room may not ...",30,3169020.0
1,34839314,https://www.airbnb.com/rooms/34839314,In white!,15,233565.0
2,42428695,https://www.airbnb.com/rooms/42428695,"Hotel Epik Montreal, Penthouse",30,210000.0
3,26400810,https://www.airbnb.com/rooms/26400810,Room Griffintown 2019 / Rent CAD$6000 for 6 mos,30,129690.0
4,43110787,https://www.airbnb.com/rooms/43110787,SUMMER SUBLET in the McGill area!,30,120000.0
5,53877418,https://www.airbnb.com/rooms/53877418,Boutique Hotel in Montreal Old PORT,30,120000.0
6,521395,https://www.airbnb.com/rooms/521395,Montreal Village Downtown Quebec,30,97470.0
7,630597748039638764,https://www.airbnb.com/rooms/630597748039638764,3 bedroom condo with massive patio,30,90000.0
8,25375230,https://www.airbnb.com/rooms/25375230,Paradis Au Coeur Du Centre Ville,30,74970.0
9,705918801669364386,https://www.airbnb.com/rooms/705918801669364386,Huge & Stylish 10 Bedroom w/Patio - Sleeps 33!,22,65978.0


### List of potential customers for an Airbnb cleaning business
These are list of Airbnb units that were rated low for cleanliness and might be potential customers for someone wanting to start up a cleaning business.
<br> The *num_lowclean_reviews* column represents the total number of ratings that were 3 and below.

In [4]:
pd.read_sql_query("""SELECT host_id, host_url, host_name, COUNT(*) AS num_lowclean_reviews FROM df
                  WHERE review_scores_cleanliness <= 3
                  GROUP BY host_id, host_url, host_name 
                  ORDER BY num_lowclean_reviews DESC""", conn)

,host_id,host_url,host_name,num_lowclean_reviews
0,38459934,https://www.airbnb.com/users/show/38459934,Corporate Stays,144
1,195649860,https://www.airbnb.com/users/show/195649860,Claude,52
2,434955825,https://www.airbnb.com/users/show/434955825,Gabrielle,33
3,438658450,https://www.airbnb.com/users/show/438658450,Lui,24
4,194001133,https://www.airbnb.com/users/show/194001133,Alexandre Et Steven,22
...,...,...,...,...
2350,478419458,https://www.airbnb.com/users/show/478419458,Antoine,1
2351,478575338,https://www.airbnb.com/users/show/478575338,Élisabeth,1
2352,478861941,https://www.airbnb.com/users/show/478861941,Bita,1
2353,478920320,https://www.airbnb.com/users/show/478920320,Cedric,1


### List of most top rated review scores

In [5]:
pd.read_sql_query("""SELECT price, host_url, host_name, COUNT(*) AS num_top_reviews FROM df
                  WHERE review_scores_rating = 5
                  GROUP BY price, host_url, host_name 
                  ORDER BY num_top_reviews DESC""", conn)

,price,host_url,host_name,num_top_reviews
0,242.0,https://www.airbnb.com/users/show/423142109,David,9
1,159.0,https://www.airbnb.com/users/show/195501369,Julio,7
2,55.0,https://www.airbnb.com/users/show/436048128,Linda,6
3,21.0,https://www.airbnb.com/users/show/17047853,Daniela,5
4,200.0,https://www.airbnb.com/users/show/469111085,Viktor,5
...,...,...,...,...
2854,2071.0,https://www.airbnb.com/users/show/138835642,Al,1
2855,3913.0,https://www.airbnb.com/users/show/448457025,Clubhaus,1
2856,4156.0,https://www.airbnb.com/users/show/448457025,Clubhaus,1
2857,5000.0,https://www.airbnb.com/users/show/7250257,Stewie,1


### List of top responders

In [6]:
pd.read_sql_query("""SELECT DISTINCT host_url, host_name
                  FROM df
                  WHERE host_response_time = "within an hour"
                  ORDER BY host_name""", conn)


,host_url,host_name
0,https://www.airbnb.com/users/show/8500362,500 Squares
1,https://www.airbnb.com/users/show/16481681,Aaliyah
2,https://www.airbnb.com/users/show/476385450,Abbas
3,https://www.airbnb.com/users/show/70447,Abdel
4,https://www.airbnb.com/users/show/422741657,Abdel
...,...,...
2605,https://www.airbnb.com/users/show/264623654,Zorhan
2606,https://www.airbnb.com/users/show/386112622,Zoélie
2607,https://www.airbnb.com/users/show/104515444,Éliane
2608,https://www.airbnb.com/users/show/355072276,Éva


### * Other additional queries *

### Price ranges in Montreal

Airbnb prices range from 13.00 to 105,634 CAD.

In [7]:
pd.read_sql_query("""SELECT DISTINCT price
                  FROM df
                  ORDER BY price""", conn)

,price
0,13.0
1,14.0
2,15.0
3,16.0
4,17.0
...,...
694,7200.0
695,12000.0
696,13294.0
697,15571.0


### List of Airbnb where host started in January 2014

In [8]:
pd.read_sql_query("""SELECT DISTINCT host_name, property_type, host_neighbourhood, price
                        FROM df
                        WHERE host_since = "2014-01-01"
                   """, conn)

,host_name,property_type,host_neighbourhood,price
0,Jessica,Entire rental unit,None,139.0


### List of all property types in the listing

In [9]:
pd.read_sql_query("""SELECT DISTINCT property_type
                  FROM df
                  ORDER BY property_type""", conn)

,property_type
0,Boat
1,Camper/RV
2,Campsite
3,Casa particular
4,Entire bed and breakfast
5,Entire bungalow
6,Entire cabin
7,Entire chalet
8,Entire condo
9,Entire cottage


### List of entire home rental with 3 beds available for less than 2 weeks

In [10]:
pd.read_sql_query("""SELECT DISTINCT property_type, price,availability_30, host_neighbourhood
                        FROM df
                        WHERE beds > 3 AND availability_30 < 14 AND property_type = "Entire home"
                   """, conn) 

,property_type,price,availability_30,host_neighbourhood
0,Entire home,399.0,5,Pointe-Sainte-Charles
1,Entire home,185.0,0,Notre-Dame-de-Grace
2,Entire home,180.0,0,Le Plateau
3,Entire home,530.0,0,Le Sud-Ouest
4,Entire home,185.0,0,La Petite-Patrie
...,...,...,...,...
188,Entire home,200.0,0,None
189,Entire home,89.0,0,Laval-des-rapides
190,Entire home,140.0,0,None
191,Entire home,495.0,2,None


### List of airbnb places specifically only in Laval, Canada with value score of 4.5 and above

In [11]:
pd.read_sql_query("""SELECT DISTINCT host_location,price, review_scores_value,host_neighbourhood
                  FROM df
                  WHERE host_location LIKE '%Laval, Canada%' AND review_scores_value >=4.5
                  ORDER BY review_scores_value DESC""", conn)

,host_location,price,review_scores_value,host_neighbourhood
0,"Laval, Canada",35.0,5.0,Le Plateau
1,"Laval, Canada",62.0,5.0,Gay Village
2,"Laval, Canada",39.0,5.0,None
3,"Laval, Canada",54.0,5.0,Rosemont-La Petite-Patrie
4,"Laval, Canada",50.0,5.0,Downtown Montreal
...,...,...,...,...
138,"Laval, Canada",268.0,4.5,Downtown Montreal
139,"Laval, Canada",416.0,4.5,Downtown Montreal
140,"Laval, Canada",193.0,4.5,None
141,"Laval, Canada",85.0,4.5,Laval-des-rapides


### The maximum and minimum number of reviews in the dataset

In [12]:
pd.read_sql_query("""SELECT DISTINCT MAX(number_of_reviews) AS Max, MIN(number_of_reviews) AS Min
                  FROM df""", conn)

,Max,Min
0,734,0


### The most and least number of guest can be accommodated

In [13]:
pd.read_sql_query("""SELECT DISTINCT MAX(accommodates) AS Max, MIN(accommodates) AS Min
                  FROM df""", conn)

,Max,Min
0,16,1


### Number of places that can instantly be booked with prices lower than 250
There are 253 places.

In [14]:
pd.read_sql_query("""
                    SELECT 
                    COUNT(instant_bookable) AS Total
                    FROM df
                    WHERE instant_bookable = "t" AND price >= 250
                    GROUP BY (instant_bookable)""", conn)

,Total
0,731


### Places with Wifi, Heating and Free Parking

In [15]:
pd.read_sql_query("""SELECT host_neighbourhood, amenities
                        FROM df
                        WHERE amenities LIKE '%Wifi%' AND amenities LIKE '%Heating%' AND amenities LIKE '%Free parking%'
                   """, conn) 

,host_neighbourhood,amenities
0,La Petite-Patrie,"[""Kitchen"", ""Refrigerator"", ""Hair dryer"", ""Fre..."
1,Hochelaga-Maisonneuve,"[""Refrigerator"", ""Microwave"", ""Carbon monoxide..."
2,Little Burgundy,"[""Refrigerator"", ""Hot tub"", ""Microwave"", ""Kitc..."
3,Pointe-Sainte-Charles,"[""Refrigerator"", ""Microwave"", ""Body soap"", ""Dr..."
4,Notre-Dame-de-Grace,"[""Kitchen"", ""Free parking on premises"", ""Dryer..."
...,...,...
2825,None,"[""Single level home"", ""Refrigerator"", ""Microwa..."
2826,None,"[""Refrigerator"", ""Wifi \u2013 20 Mbps"", ""Kitch..."
2827,None,"[""Single level home"", ""Refrigerator"", ""Microwa..."
2828,Laval-des-rapides,"[""Refrigerator"", ""Microwave"", ""Carbon monoxide..."


### Names of all Entire townhouse in Notre-Dame-de-Grace

In [16]:
pd.read_sql_query("""SELECT DISTINCT name, price
                        FROM df
                        WHERE host_neighbourhood = "Notre-Dame-de-Grace" AND property_type = "Entire townhouse"
                        ORDER BY review_scores_checkin
                   """, conn)

,name,price
0,Bright and stylish home in trendy Monkland Vil...,295.0
1,Newly fully renovated cosy townhouse in great ...,2000.0
2,Center of town - 5 lovely and airy rooms with ...,150.0
3,"CentreVille - JardinBBCue 3ch/1,5sdb, MétroBea...",155.0
4,"Beautiful, Charming 1920s Renovated House near...",146.0
5,VIP PENTHOUSE OUTREMONT DOWNTOWN MONTREAL,2000.0
6,"NDG -Lower Duplex 4 bd/3 bath Solarium,Villa M...",150.0
7,NDG-Bright Top Duplex 4bedrooms/2bathrooms Ter...,125.0
8,Beautiful house to rent Montreal ND,450.0
9,"Very nice house, pleasant neighborhood life, m...",400.0


## Conclusion

All the goals for the SQL project were met and we were clearly able to distinguish the top most profitable Airbnb accommodations in Montréal based on their availability in a month. We were able to gather a list of all accommodations with poor ratings for cleanliness that might be flare a potential start-up for entrepreneurs who are looking into invest in complementary businesses related to Airbnb. We were able to filter the top most rated listings for clients who are looking for 5 star accommodations. Also, we were able to add more queries that might possibly help customers in their decision making.